<a href="https://colab.research.google.com/github/ishita1611/Task-2/blob/main/allcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y transformers
!pip install transformers==4.52.4 datasets peft accelerate langgraph --quiet



Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4


In [2]:
!pip install -U fsspec==2023.6.0 --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.6.0 which is incompatible.


In [3]:
!pip uninstall -y transformers
!pip uninstall -y huggingface_hub
!pip uninstall -y peft
!pip uninstall -y accelerate


Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
Found existing installation: huggingface-hub 0.33.0
Uninstalling huggingface-hub-0.33.0:
  Successfully uninstalled huggingface-hub-0.33.0
Found existing installation: peft 0.15.2
Uninstalling peft-0.15.2:
  Successfully uninstalled peft-0.15.2
Found existing installation: accelerate 1.8.1
Uninstalling accelerate-1.8.1:
  Successfully uninstalled accelerate-1.8.1


In [4]:
!pip install transformers==4.39.3 peft==0.10.0 accelerate datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 76.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.39.3 which is incompatible.


In [1]:
import transformers
print(transformers.__version__)  # Should print 4.39.3


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

4.39.3


In [3]:
# Package versions check
import transformers
print("Transformers version:", transformers.__version__)  # Should show 4.52.4

# Load dataset
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig
import torch

dataset = load_dataset("imdb")
train_data = dataset["train"].shuffle(42).select(range(2000))
test_data = dataset["test"].shuffle(42).select(range(1000))

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)
train_data = train_data.map(tokenize, batched=True)
test_data = test_data.map(tokenize, batched=True)
train_data.set_format(type="torch", columns=["input_ids","attention_mask","label"])
test_data.set_format(type="torch", columns=["input_ids","attention_mask","label"])

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
lora_config = LoraConfig(
    r=8, lora_alpha=16, target_modules=["q_lin", "v_lin"],
    lora_dropout=0.05, bias="none", task_type="SEQ_CLS"
)
model = get_peft_model(model, lora_config)

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./peft_imdb_lora",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="no",
    evaluation_strategy="no"
)


Transformers version: 4.39.3


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('./finetuned_model/tokenizer_config.json',
 './finetuned_model/special_tokens_map.json',
 './finetuned_model/vocab.txt',
 './finetuned_model/added_tokens.json',
 './finetuned_model/tokenizer.json')

In [6]:
from langchain_core.runnables import RunnableLambda


In [1]:
!pip uninstall -y transformers huggingface_hub peft accelerate langgraph
!pip install transformers==4.39.3 peft==0.10.0 accelerate datasets langgraph --quiet


Found existing installation: transformers 4.39.3
Uninstalling transformers-4.39.3:
  Successfully uninstalled transformers-4.39.3
Found existing installation: huggingface-hub 0.33.1
Uninstalling huggingface-hub-0.33.1:
  Successfully uninstalled huggingface-hub-0.33.1
Found existing installation: peft 0.10.0
Uninstalling peft-0.10.0:
  Successfully uninstalled peft-0.10.0
Found existing installation: accelerate 1.8.1
Uninstalling accelerate-1.8.1:
  Successfully uninstalled accelerate-1.8.1
Found existing installation: langgraph 0.4.10
Uninstalling langgraph-0.4.10:
  Successfully uninstalled langgraph-0.4.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.39.3 which is incompatible.


In [1]:
import torch
import torch.nn.functional as F
import logging
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from langgraph.graph import StateGraph
from langchain_core.runnables import RunnableLambda
from datetime import datetime

# === SETUP ===
logging.basicConfig(
    filename="classifier.log",
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your fine-tuned model
model_path = "./finetuned_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)

label_map = model.config.id2label if hasattr(model.config, "id2label") else {0: "NEGATIVE", 1: "POSITIVE"}
CONFIDENCE_THRESHOLD = 0.75

# === NODES ===

def inference_node(state):
    text = state.get("text", "").strip()
    if not text:
        raise ValueError("No text provided.")

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
    probs = F.softmax(logits, dim=1)
    conf, pred = torch.max(probs, dim=1)
    confidence = conf.item()
    prediction = label_map.get(pred.item(), str(pred.item()))

    logging.info(f"[InferenceNode] Input: {text} | Prediction: {prediction} | Confidence: {confidence:.2f}")

    state.update({
        "prediction": prediction,
        "confidence": confidence
    })
    return state

def confidence_check_node(state):
    confidence = state["confidence"]
    if confidence >= CONFIDENCE_THRESHOLD:
        state["route"] = "accept"
        verdict = "ACCEPT"
    else:
        state["route"] = "fallback"
        verdict = "FALLBACK"
        logging.info(f"[ConfidenceCheckNode] Low confidence ({confidence:.2f}). Triggering fallback.")
    return state

def fallback_node(state):
    print("\n AHH!!Not confident in prediction.")
    print(f"Text: {state['text']}")
    print(f"Predicted: {state['prediction']} (Confidence: {state['confidence']:.2f})")
    correction = input("Could you clarify? What should the label actually be? ").strip()
    logging.info(f"[FallbackNode] User corrected to: {correction}")
    state["prediction"] = correction
    return state

def final_output_node(state):
    conf = state["confidence"]
    label = state["prediction"]

    # Human-friendly interpretation
    if conf < 0.40:
        mood = "😬 Bad"
    elif conf < 0.60:
        mood = "😐 Okay"
    elif conf < 0.80:
        mood = "🙂 Good"
    else:
        mood = "🤩 Excellent"

    print(f"\nFinal Label: {label} (Confidence: {conf:.2f}) {mood}")
    logging.info(f"[FINAL] Final label: {label} | Confidence: {conf:.2f} ({mood})")
    return state

# === GRAPH ===

graph = StateGraph(dict)
graph.add_node("inference", RunnableLambda(inference_node))
graph.add_node("confidence_check", RunnableLambda(confidence_check_node))
graph.add_node("fallback", RunnableLambda(fallback_node))
graph.add_node("final", RunnableLambda(final_output_node))

graph.set_entry_point("inference")
graph.add_edge("inference", "confidence_check")
graph.add_conditional_edges("confidence_check", {
    "accept": RunnableLambda(final_output_node),
    "fallback": RunnableLambda(fallback_node)
})
graph.add_edge("fallback", "final")
graph.set_finish_point("final")

app = graph.compile()

# === CLI ===

try:
    while True:
        user_input = input("\n📥 Input (or 'exit'): ").strip()
        if user_input.lower() == "exit":
            print("Bye!")
            break
        app.invoke({"text": user_input})
except KeyboardInterrupt:
    print("\n Oopss!! Interrupted. Stay royal!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_


📥 Input (or 'exit'): I love the movie

Final Label: LABEL_1 (Confidence: 0.57) 😐 Okay

 AHH!!Not confident in prediction.
Text: I love the movie
Predicted: LABEL_1 (Confidence: 0.57)
Could you clarify? What should the label actually be? negative

 Oopss!! Interrupted. Stay royal!
